In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "chicago_taxi_trips" dataset
dataset_ref = client.dataset("chicago_taxi_trips", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "taxi_trips" table
table_ref = dataset_ref.table("taxi_trips")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.


,unique_key,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,...,extras,trip_total,payment_type,company,pickup_latitude,pickup_longitude,pickup_location,dropoff_latitude,dropoff_longitude,dropoff_location
0,d7142baead61226e1f0d5fbb42053148d4ea1234,83509dee15087de780f4ceef090a6d37029553d45b193c...,2013-12-28 01:00:00+00:00,2013-12-28 01:00:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,8.85,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
1,c4b5ecbc1199c68bc7c7336502d51c7af3a6f6dd,d6e1a9e103336c396201abe9ceb00795fcd41e14ccbf54...,2014-01-07 10:00:00+00:00,2014-01-07 09:45:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,7.74,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None
2,5122fd569680033efac6d8d1990fa60014f112b7,23d5d31d196f02492dee2d0a37f4f52cce7904302213d4...,2014-07-01 20:15:00+00:00,2014-07-01 20:15:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,27.00,Credit Card,Park Ridge Taxi and Livery,NaN,NaN,None,NaN,NaN,None
3,556af535d64bc6460244a5bad92966fd5d85421f,442f4c9dd3dde549a82c2dc0ef65b85e54664aa28f5b55...,2014-01-07 13:30:00+00:00,2014-01-07 13:30:00+00:00,0.0,0.0,NaN,NaN,NaN,NaN,...,0.0,7.65,Credit Card,Taxi Affiliation Services,NaN,NaN,None,NaN,NaN,None
4,bb1f95719e77f9c887912b7b2ac4aa4b4b1a6bcb,01cfdf564af1a522661ba984b93574d481b54e27516833...,2014-01-25 21:15:00+00:00,2014-01-25 21:00:00+00:00,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,11.45,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,NaN,NaN,None,NaN,NaN,None


### 1) Predict the demand for taxis

In [3]:
avg_num_trips_query = """
                      WITH trips_by_day AS
                      (
                      SELECT DATE(trip_start_timestamp) AS trip_date,
                          COUNT(*) as num_trips
                      FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                      WHERE trip_start_timestamp >= '2016-01-01' AND trip_start_timestamp < '2018-01-01'
                      GROUP BY trip_date
                      ORDER BY trip_date
                      )
                      SELECT trip_date,
                          AVG(num_trips) OVER (ORDER BY trip_date
                          ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING) AS avg_num_trips
                      FROM trips_by_day
                      """


/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,trip_date,avg_num_trips
0,2016-01-01,80461.937500
1,2016-01-02,80150.647059
2,2016-01-03,79419.611111
3,2016-01-04,79810.421053
4,2016-01-05,80293.900000


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### Separate and order trips by community area

In [5]:
trip_number_query = """
                    SELECT pickup_community_area,
                        trip_start_timestamp,
                        trip_end_timestamp,
                        RANK() OVER (PARTITION BY pickup_community_area 
                        ORDER BY trip_start_timestamp) AS trip_number
                    FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                    WHERE DATE(trip_start_timestamp) = '2017-05-01'
                    """

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,pickup_community_area,trip_start_timestamp,trip_end_timestamp,trip_number
0,39.0,2017-05-01 01:15:00+00:00,2017-05-01 01:30:00+00:00,1
1,39.0,2017-05-01 02:15:00+00:00,2017-05-01 02:45:00+00:00,2
2,39.0,2017-05-01 05:15:00+00:00,2017-05-01 05:30:00+00:00,3
3,39.0,2017-05-01 05:45:00+00:00,2017-05-01 06:15:00+00:00,4
4,39.0,2017-05-01 06:45:00+00:00,2017-05-01 07:00:00+00:00,5


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### Time elapses between trips
![first_commands](https://i.imgur.com/qjvQzg8.png)

In [7]:
break_time_query = """
                   SELECT taxi_id,
                       trip_start_timestamp,
                       trip_end_timestamp,
                       TIMESTAMP_DIFF(
                           trip_start_timestamp, 
                           LAG(trip_end_timestamp,1) 
                               OVER (
                                    PARTITION BY taxi_id 
                                    ORDER BY trip_start_timestamp), 
                           MINUTE) as prev_break
                   FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
                   WHERE DATE(trip_start_timestamp) = '2017-05-01' 
                   """

/opt/conda/lib/python3.7/site-packages/google/cloud/bigquery/client.py:440: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


,taxi_id,trip_start_timestamp,trip_end_timestamp,prev_break
0,00ce2d9c23209022f971cf517dbf6db5dec1bd49464e47...,2017-05-01 00:45:00+00:00,2017-05-01 01:00:00+00:00,NaN
1,00ce2d9c23209022f971cf517dbf6db5dec1bd49464e47...,2017-05-01 04:15:00+00:00,2017-05-01 04:30:00+00:00,195.0
2,00ce2d9c23209022f971cf517dbf6db5dec1bd49464e47...,2017-05-01 04:45:00+00:00,2017-05-01 04:45:00+00:00,15.0
3,03e4655e53ac613cde9831459d9ad728f0890ece70076f...,2017-05-01 04:15:00+00:00,2017-05-01 04:30:00+00:00,NaN
4,03e4655e53ac613cde9831459d9ad728f0890ece70076f...,2017-05-01 04:15:00+00:00,2017-05-01 04:15:00+00:00,-15.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>